<a href="https://colab.research.google.com/github/richardjana/freeCodeCamp/blob/main/Machine_Learning_with_Python/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tensorflow==2.15.0 tensorflow-datasets
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import string

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train = pd.read_csv(train_file_path, sep='\t', header=None, names=['type', 'message'])
test = pd.read_csv(test_file_path, sep='\t', header=None, names=['type', 'message'])
train.head()

for df in [train, test]:
  df['type'] = df['type'].map({'ham': 0, 'spam': 1}).astype(int)

In [ ]:
# clean message texts
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

train['message'] = train['message'].apply(clean_text)
test['message'] = test['message'].apply(clean_text)

In [ ]:
# tokenize messages
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train['message'])
X_test = vectorizer.transform(test['message'])

y_train = train['type']
y_test = test['type']

In [ ]:
# train model
clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  label_map = {0: "ham",
               1: "spam"}

  proba = clf.predict_proba(vectorizer.transform([clean_text(pred_text)]))

  return [proba[0][1], label_map[np.argmax(proba[0])]]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
